# RNN - AAPL

## `Pandas Reader`

In [1]:
''' Linear Algebra '''
import pandas as pd
import numpy as np

'''Data visualization'''
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")

/Users/davywai/.pyenv/versions/3.7.6/envs/lewagon/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
''' Read financial data'''
from pandas_datareader.data import DataReader

''' Handling time'''
from datetime import datetime

In [3]:
# tech companies

tech_list = ['AAPL','GOOG','MSFT', 'AMZN']

In [4]:
# today

end = datetime.now()
end

datetime.datetime(2021, 6, 3, 18, 8, 23, 634824)

In [5]:
today.year

NameError: name 'today' is not defined

In [ ]:
# two years ago

start = datetime(end.year-2, end.month, end.day)
start

In [ ]:
stock_prices = {}

for company in tech_list:
    stock_prices[company] = DataReader(company, "yahoo", start,end)

In [ ]:
type(stock_prices)

In [ ]:
for k,v in stock_prices.items():
    print("-"*50)
    print(k)
    print(v.shape)
    display(v.head(3))

In [ ]:
for k,v in stock_prices.items():
    v["company_name"] = k

In [ ]:
data = pd.concat([stock_prices["AAPL"],stock_prices["GOOG"],stock_prices["MSFT"],stock_prices["AMZN"]],axis=0)

In [ ]:
data.sample(10)

## Data Visualisation

### Historical Prices

In [ ]:
# Let's see a historical view of the closing price


plt.figure(figsize=(15, 6))


for i, company in enumerate(tech_list, 1):
    plt.subplot(2, 2, i)
    stock_prices[company]['Adj Close'].plot()
    plt.ylabel('Adj Close')
    plt.xlabel(None)
    plt.title(f"Closing Price of {tech_list[i - 1]}")
    
plt.tight_layout()

### Volumes Traded

In [ ]:
# Let's see a historical view of the closing price


plt.figure(figsize=(15, 6))


for i, company in enumerate(tech_list, 1):
    plt.subplot(2, 2, i)
    stock_prices[company]['Volume'].plot()
    plt.ylabel('Volume')
    plt.xlabel(None)
    plt.title(f"Volume {tech_list[i - 1]}")
    
plt.tight_layout()

### Daily returns

In [ ]:
stock_prices["AAPL"]

In [ ]:
(44.083302-42.527481)/42.527481

In [ ]:
stock_prices["AAPL"]["Adj Close"].pct_change()

In [ ]:
for k,v in stock_prices.items():
    v["daily_pct_change"] = v["Adj Close"].pct_change()

In [ ]:
for k,v in stock_prices.items():
    print("-"*50)
    print(f"pct_change distribution for {k}")
    sns.displot(v["daily_pct_change"])
    plt.show();

### Comparing tech companies

In [ ]:
DataReader(tech_list, "yahoo", start,end)

In [ ]:
new_data = DataReader(tech_list, "yahoo", start,end)["Adj Close"]
new_data

In [ ]:
daily_returns = new_data.pct_change()
daily_returns

In [ ]:
sns.pairplot(daily_returns)

In [ ]:
daily_returns.corr()

In [ ]:
sns.heatmap(daily_returns.corr(),annot=True, cmap="RdYlBu")

## AAPL Stock Price

In [ ]:
apple = stock_prices['AAPL']
apple.tail(5)

In [ ]:
apple = stock_prices['AAPL']["Adj Close"]
apple

In [ ]:
apple = stock_prices['AAPL']["Adj Close"]
apple

In [ ]:
apple.shape

In [ ]:
apple = stock_prices['AAPL'][["Adj Close"]]
apple

In [ ]:
apple.shape

In [ ]:
plt.figure(figsize=(16,6))
plt.title('Close Price History')
plt.plot(apple['Adj Close'])
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.show()

### Temporal Train-Test Split

In [ ]:
percentage_split = 0.80

In [ ]:
len(apple)*percentage_split

In [ ]:
int(len(apple)*percentage_split)

In [ ]:
aapl_train = apple[:int(len(apple)*percentage_split)]
aapl_test = apple[int(len(apple)*percentage_split):]

### Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
minmax_scaler.fit(aapl_train)

aapl_train_transformed = minmax_scaler.transform(aapl_train)
aapl_test_transformed = minmax_scaler.transform(aapl_test)

In [ ]:
minmax_scaler.data_min_

In [ ]:
minmax_scaler.data_max_

### Creating subsequences...

**Draft...**

In [ ]:
length = 10
horizon = 5

In [ ]:
last_possible_index = len(aapl_train) - length - horizon
last_possible_index

In [ ]:
random_start = np.random.choice(last_possible_index)
random_start

In [ ]:
random_start + length

In [ ]:
random_start + length + horizon

In [ ]:
one_subsequence = aapl_train.iloc[random_start:random_start + length]
one_target = aapl_train.iloc[random_start + length + horizon]

In [ ]:
one_subsequence

In [ ]:
one_target

`subsequence`

In [ ]:
def subsequence(df, length, horizon):
    last_possible_index = len(df) - length - horizon
    random_start = np.random.choice(last_possible_index)
    X = df[random_start:random_start + length]
    y = df[random_start + length + horizon][0]
    return X,y
    
    

`multiple subsequences`

In [ ]:
def multiple_subsequences(df, nb_subsequences, horizon = 5):
    
    list_of_X = []
    list_of_y = []
    
    
    list_of_lengths = np.random.randint(10,15,nb_subsequences)
    
    for length in list_of_lengths:
        tmp_X, tmp_y = subsequence(df, length, horizon)
        list_of_X.append(tmp_X)
        list_of_y.append(tmp_y)
        
    return list_of_X, list_of_y

In [ ]:
multiple_subsequences(aapl_train_transformed, 3, horizon = 5)[1]

## LSTM

### Creating train_test_data

In [ ]:
X_train, y_train = multiple_subsequences(aapl_train_transformed, 2000, horizon = 5)
X_test, y_test = multiple_subsequences(aapl_test_transformed, 1000, horizon = 5)

In [ ]:
type(X_train)

In [ ]:
len(X_train)

In [ ]:
# [len(X_train[k]) for k in range(len(X_train)]

sns.histplot([len(_) for _ in X_train],bins=5);

In [ ]:
sns.histplot([len(_) for _ in X_test],bins=5);

### Padding

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X_train_pad = pad_sequences(X_train, padding = "post", dtype="float32", value = -999)
X_test_pad = pad_sequences(X_test, padding = "post", dtype="float32", value = -999)

In [ ]:
sns.histplot([len(_) for _ in X_train_pad],bins=5);

In [ ]:
sns.histplot([len(_) for _ in X_test_pad],bins=5);

In [ ]:
X_train_pad.shape

In [ ]:
X_test_pad.shape

In [ ]:
X_train_pad[0]

### Architecture

In [ ]:
from tensorflow.keras import models, layers


model = models.Sequential()

model.add(layers.Masking(mask_value = -999))

model.add(layers.LSTM(units = 32, activation="tanh", return_sequences= True))
model.add(layers.LSTM(units = 16, activation="tanh", return_sequences= True))
model.add(layers.LSTM(units = 8, activation="tanh", return_sequences= False))

model.add(layers.Dense(8,activation="relu"))


model.add(layers.Dense(1,activation="relu")) # or linear 

In [ ]:
# model.summary()

### Compile

In [ ]:
from tensorflow.keras import metrics

my_mae  = metrics.MeanAbsoluteError()

In [ ]:
model.compile(loss="mse",
             optimizer="rmsprop",
             metrics = [my_mae])

### Fit

In [ ]:
type(y_train)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience = 10, restore_best_weights = True)


history = model.fit(X_train_pad, np.array(y_train),
                   validation_split = 0.3,
                   batch_size = 8,
                   epochs = 100,
                   callbacks=[es],
                   verbose = 1)

### Evaluate

In [ ]:
res = model.evaluate(X_test_pad, np.array(y_test), verbose = 1)

### Predictions

In [ ]:
predictions = model.predict(X_test_pad)

The operation computed during a `MinMaxScaler` Operation is:

$$x_{scaled} = \frac{x - x_{min}}{x_{max}- x_{min}}$$

To descale it:

$$x = x_{scaled} * (x_{max}- x_{min}) + x_{min}$$

In [ ]:
x_min = minmax_scaler.data_min_
x_min

In [ ]:
x_max = minmax_scaler.data_max_
x_max

In [ ]:
unscaled_predictions = predictions * (x_max - x_min) + x_min

In [ ]:
unscaled_predictions = minmax_scaler.inverse_transform(predictions)

In [ ]:
unscaled_predictions

In [ ]:
unscaled_y_test = minmax_scaler.inverse_transform(np.array([y_test]))

In [ ]:
unscaled_y_test

In [ ]:
np.mean(np.abs(unscaled_y_test - unscaled_predictions))